In [1]:
import numpy as np

import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchsummary import summary

from trainers import trainTorch, advTrain
from evalAttacks import evalClean
from evalAttacks import CWAttackRandomEnsem, CWAttackEnsem

/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/iis-cvl/anaconda3/envs/advAt

/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/iis-cvl

In [2]:
nb_epochs = 2
batch_size = 128
learning_rate = 0.001
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=True, download=True,
                    transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size)

Files already downloaded and verified


In [3]:
# Loading models from modelZoo
from modelZooCifar import CIFARNet, ensembleModel, ensembleModel4
CiNet1 = CIFARNet()
CiNet2 = CIFARNet()
CiNet3 = CIFARNet()
CiNet4 = CIFARNet()

ckptPath = 'cifarCkpts/'

ensemNet = ensembleModel(CiNet1, CiNet2, CiNet3)
ensemNet4 = ensembleModel4(CiNet1, CiNet2, CiNet3, CiNet4)

CiNet1.load_state_dict(torch.load(ckptPath + 'CIFARNet1.pth'))
CiNet2.load_state_dict(torch.load(ckptPath + 'CIFARNet2.pth'))
CiNet3.load_state_dict(torch.load(ckptPath + 'CIFARNet3.pth'))
CiNet4.load_state_dict(torch.load(ckptPath + 'CIFARNet4.pth'))
ensemNet.load_state_dict(torch.load(ckptPath + 'ensemNet.pth'))
ensemNet4.load_state_dict(torch.load(ckptPath + 'ensemNet4.pth'))

if torch.cuda.is_available():
    CiNet1 = CiNet1.cuda()
    CiNet2 = CiNet2.cuda()
    CiNet3 = CiNet3.cuda()
    CiNet4 = CiNet4.cuda()
    ensemNet = ensemNet.cuda()
    ensemNet4 = ensemNet4.cuda()


In [4]:
# Evaluate all models on clean original test data
evalClean(model1=CiNet1, test_loader=test_loader, singleModel=1)
evalClean(model1=CiNet2, test_loader=test_loader, singleModel=1)
evalClean(model1=CiNet3, test_loader=test_loader, singleModel=1)
evalClean(model1=ensemNet, test_loader=test_loader, singleModel=1)
evalClean(model1=ensemNet4, test_loader=test_loader, singleModel=1)

Evaluating single model results on clean data
Clean accuracy: 75.48%
Evaluating single model results on clean data
Clean accuracy: 73.84%
Evaluating single model results on clean data
Clean accuracy: 74.93%
Evaluating single model results on clean data
Clean accuracy: 78.05%
Evaluating single model results on clean data
Clean accuracy: 77.86%


In [5]:
# Evaluate the attack against each other and against 
modelGroup = [CiNet1, CiNet2, CiNet3]
CWAttackEnsem(CiNet1, modelGroup, test_loader)
CWAttackEnsem(CiNet2, modelGroup, test_loader)
CWAttackEnsem(CiNet3, modelGroup, test_loader)

On Batch 1
Running on the original Ensembled
On Batch 2
Running on the original Ensembled
Adv accuracy for original ensem model: 0.000％
Adv accuracy for test model 0: 0.000%
Adv accuracy for test model 1: 69.531%
Adv accuracy for test model 2: 73.047%
Adv accuracy for Ensemble Voting: 62.500％
On Batch 1
Running on the original Ensembled
On Batch 2
Running on the original Ensembled
Adv accuracy for original ensem model: 0.000％
Adv accuracy for test model 0: 71.094%
Adv accuracy for test model 1: 0.000%
Adv accuracy for test model 2: 72.656%
Adv accuracy for Ensemble Voting: 62.891％
On Batch 1
Running on the original Ensembled
On Batch 2
Running on the original Ensembled
Adv accuracy for original ensem model: 0.000％
Adv accuracy for test model 0: 69.922%
Adv accuracy for test model 1: 68.359%
Adv accuracy for test model 2: 0.000%
Adv accuracy for Ensemble Voting: 60.938％


In [7]:
CWAttackEnsem(ensemNet, modelGroup, test_loader)

On Batch 1
Running on the original Ensembled
On Batch 2
Running on the original Ensembled
Adv accuracy for original ensem model: 0.000％
Adv accuracy for test model 0: 47.266%
Adv accuracy for test model 1: 50.000%
Adv accuracy for test model 2: 55.469%
Adv accuracy for Ensemble Voting: 56.641％


In [8]:
CWAttackRandomEnsem(ensemNet4, [CiNet1, CiNet2, CiNet3, CiNet4], test_loader)

On Batch 1
Running on the original Ensembled
Running on the random Ensembled
On Batch 2
Running on the original Ensembled
Running on the random Ensembled
Adv accuracy for original ensem model: 0.000％
Adv accuracy for tested model 1: 59.766％


In [23]:
# from modelZooCifar import ensembleModel4, freeze_models, unfreeze_models
# pth = 'cifarCkpts/ensemNet4.pth'
# target_model = ensembleModel4(CiNet1, CiNet2, CiNet3, CiNet4)
# target_model = target_model.cuda()
# freeze_models(target_model)
# # opt_model = torch.optim.Adam(target_model.parameters(), lr=0.001)
# opt_model = torch.optim.SGD(filter(lambda p: p.requires_grad==True, target_model.parameters()), lr=0.01)
# unfreeze_models(target_model)

# target_model.train()
# epochs = 20
# for epoch in range(epochs):
#     loss_epoch = 0
#     if epoch == 25:
# #         opt_model = torch.optim.Adam(target_model.parameters(), lr=0.0001)
#         opt_model = torch.optim.SGD(target_model.parameters(), lr=0.005, momentum=0.9)
#     for i, data in enumerate(train_loader, 0):
#         train_imgs, train_labels = data
#         train_imgs, train_labels = train_imgs.cuda(), train_labels.cuda()
#         logits_model = target_model(train_imgs)
#         loss_model = F.cross_entropy(logits_model, train_labels)
#         loss_epoch += loss_model
#         opt_model.zero_grad()
#         loss_model.backward()
#         opt_model.step()
    
#     print('loss in epoch %d: %f' % (epoch, loss_epoch.item()))
#     if (epoch+1) % 5 == 0:
#         num_correct = 0
#         total = 0
#         for i, data in enumerate(test_loader, 0):
#             test_img, test_label = data
#             test_img, test_label = test_img.cuda(), test_label.cuda()
#             pred_lab = torch.argmax(target_model(test_img), 1)
#             num_correct += torch.sum(pred_lab==test_label,0)
#             total += len(pred_lab)
#         print('accuracy in testing set: %f\n'%(num_correct.item()/total))



# # save model
# targeted_model_file_name = pth
# torch.save(target_model.state_dict(), targeted_model_file_name)

loss in epoch 0: 309.554169
loss in epoch 1: 1.130933
loss in epoch 2: 0.604526
loss in epoch 3: 0.401626
loss in epoch 4: 0.303445
accuracy in testing set: 0.775300

loss in epoch 5: 0.243454
loss in epoch 6: 0.204142
loss in epoch 7: 0.175629
loss in epoch 8: 0.154281
loss in epoch 9: 0.137134
accuracy in testing set: 0.776400

loss in epoch 10: 0.123705
loss in epoch 11: 0.112617
loss in epoch 12: 0.103254
loss in epoch 13: 0.095507
loss in epoch 14: 0.088910
accuracy in testing set: 0.777500

loss in epoch 15: 0.083000
loss in epoch 16: 0.077901
loss in epoch 17: 0.073458
loss in epoch 18: 0.069418
loss in epoch 19: 0.065809
accuracy in testing set: 0.778600

